In [37]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn import metrics
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import precision_recall_curve
from sklearn.metrics import PrecisionRecallDisplay
from sklearn.naive_bayes import GaussianNB
from sklearn.model_selection import GridSearchCV
from sklearn import preprocessing
from sklearn.ensemble import RandomForestClassifier
from sklearn.datasets import make_classification
df_raw = pd.read_csv('BankChurners.csv')

y = df_raw['Attrition_Flag'].to_numpy()

#===========================================
cols = df_raw.describe().columns
cols = cols.tolist()
dfx = df_raw[cols[1:-2]]
X_num = dfx.to_numpy()
print(str(X_num.shape[1]) + ' Numerical Only')

# categorical, Gender
df_gender = pd.get_dummies(df_raw['Gender'])
genF = df_gender['F'].to_numpy()
Xf = np.append(X_num,genF.reshape((10127,1)),axis=1)
print(str(Xf.shape[1]) + ' with Sex')

# categorical, Education
# 0 = unknown, 1 = undeducated, ... , 6 = Doctorate
# But maybe less educated use credit card more or less than highly educated this is for churn not ability to pay
edu = df_raw['Education_Level'].unique()
df_education = pd.get_dummies(df_raw['Education_Level'])
df_education['Unknown'] = 0*df_education['Unknown']
df_education['High School'] = 2*df_education['High School']
df_education['College'] = 3*df_education['College']
df_education['Graduate'] = 4*df_education['Graduate']
df_education['Post-Graduate'] = 5*df_education['Post-Graduate']
df_education['Doctorate'] = 6*df_education['Doctorate']
df_edu = df_education.sum(axis=1)
edu = df_edu.to_numpy()
Xed = np.append(Xf, edu.reshape((10127,1)),axis=1)
print(str(Xed.shape[1]) + ' with Education')

# categorical, Marriage
# Put 4 columns because is divorced more likely than single to use this card, I dont think so
mar = df_raw['Marital_Status'].unique()
# print(mar)
df_mar = pd.get_dummies(df_raw['Marital_Status'])
df_mar.head()
mar = df_mar.to_numpy()
# print(mar.shape)
Xmar = np.append(Xed,mar,axis=1)
print(str(Xmar.shape[1]) + ' with Marriage, should add 4')

# categorical, Income
# Put 6 columns because i am not sure if this should be ordinal but I am saying not.
inc = df_raw['Income_Category'].unique()
df_inc = pd.get_dummies(df_raw['Income_Category'])
df_inc.head()
inc = df_inc.to_numpy()
Xinc = np.append(Xmar,inc,axis=1)
print(str(Xinc.shape[1]) + ' with Income, should add 6')

# categorical, Card Category
# Put 6 columns because i am not sure if this should be ordinal but I am saying not, dont know what the order is.
cc = df_raw['Card_Category'].unique()
df_cc = pd.get_dummies(df_raw['Card_Category'])
df_cc.tail()
cc = df_cc.to_numpy()
Xcc = np.append(Xinc,cc,axis=1)
print(str(Xcc.shape[1]) + ' with Card Color, should add 4')


# Final X, just rename
X = Xcc
print(X.shape)

# TODO
# Education Level ---- DONE
# Marital Status ----- DONE
# Income Category ---- DONE
# Card Category ------ DONE

14 Numerical Only
15 with Sex
16 with Education
20 with Marriage, should add 4
26 with Income, should add 6
30 with Card Color, should add 4
(10127, 30)


<h3>First Model is the Random Forest<h3>

In [38]:
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.25,random_state=0)
scaler = preprocessing.StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.fit_transform(X_test)

clf = RandomForestClassifier(max_depth=2, random_state=0)
clf.fit(X_test, y_test)
y_pred = clf.predict(X_test)
print("Recall:",metrics.recall_score(y_test,y_pred, average=None))
print(confusion_matrix(y_test, y_pred))
print(classification_report(y_test,y_pred))

Recall: [0.04947917 0.99953445]
[[  19  365]
 [   1 2147]]
                   precision    recall  f1-score   support

Attrited Customer       0.95      0.05      0.09       384
Existing Customer       0.85      1.00      0.92      2148

         accuracy                           0.86      2532
        macro avg       0.90      0.52      0.51      2532
     weighted avg       0.87      0.86      0.80      2532



This was especially terrible, 5% is not a good result

<h3>The second model is the Naive Bayes Classifier<h3>

In [39]:
gnb = GaussianNB()
y_pred = gnb.fit(X_train, y_train).predict(X_test)

print("Recall:",metrics.recall_score(y_test,y_pred, average=None))
print(confusion_matrix(y_test, y_pred))
print(classification_report(y_test,y_pred))

Recall: [0.61979167 0.92504655]
[[ 238  146]
 [ 161 1987]]
                   precision    recall  f1-score   support

Attrited Customer       0.60      0.62      0.61       384
Existing Customer       0.93      0.93      0.93      2148

         accuracy                           0.88      2532
        macro avg       0.76      0.77      0.77      2532
     weighted avg       0.88      0.88      0.88      2532



This is the best result so far, still not good

<h3>The third model is the Naive Bayes Classifier with Hyperparametrization<h3>

In [40]:
gnb.get_params()

{'priors': None, 'var_smoothing': 1e-09}

In [41]:
param_grid_nb = {'var_smoothing': np.logspace(0,-9, num=100)}

nbModel_grid = GridSearchCV(estimator=GaussianNB(), param_grid=param_grid_nb, verbose=1, cv=10, n_jobs=-1)
nbModel_grid.fit(X_train, y_train)
print(nbModel_grid.best_estimator_)

Fitting 10 folds for each of 100 candidates, totalling 1000 fits
GaussianNB(var_smoothing=0.657933224657568)


In [42]:
y_pred = nbModel_grid.predict(X_test)

print("Recall:",metrics.recall_score(y_test,y_pred, average=None))
print(confusion_matrix(y_test, y_pred))
print(classification_report(y_test,y_pred))

Recall: [0.48697917 0.97067039]
[[ 187  197]
 [  63 2085]]
                   precision    recall  f1-score   support

Attrited Customer       0.75      0.49      0.59       384
Existing Customer       0.91      0.97      0.94      2148

         accuracy                           0.90      2532
        macro avg       0.83      0.73      0.77      2532
     weighted avg       0.89      0.90      0.89      2532



I got the same, funny that when I use var_smoothing=0.658 like result from In[36] I get a worse answer

<h3>Fourth is the Logistic Regression<h3>

In [43]:
logreg = LogisticRegression()
y_pred = logreg.fit(X_train,y_train).predict(X_test)

print("Recall:",metrics.recall_score(y_test,y_pred, average=None))
print(confusion_matrix(y_test, y_pred))
print(classification_report(y_test,y_pred))

Recall: [0.59895833 0.95670391]
[[ 230  154]
 [  93 2055]]
                   precision    recall  f1-score   support

Attrited Customer       0.71      0.60      0.65       384
Existing Customer       0.93      0.96      0.94      2148

         accuracy                           0.90      2532
        macro avg       0.82      0.78      0.80      2532
     weighted avg       0.90      0.90      0.90      2532



Not very good, can I make it better>

<h3>Fifth is the Logistic Regression with Hyperparameterization<h3>

In [48]:
# Maybe I need to make my own scorer, if not disregard this cell

# Look to the scoring mechanism
# A - make_scorer Factory
# B - https://scikit-learn.org/stable/modules/model_evaluation.html#scoring
#                 Create a custom recall scoring metric

from sklearn.metrics import make_scorer
rcl_scorer = make_scorer(recall_score)#, beta=2)
rcl_scorer

make_scorer(recall_score)

In [49]:
parameters = {
    'penalty' : ['none','l2','l1'], 
    'C'       : np.logspace(-3,3,7),
    'solver'  : ['newton-cg', 'lbfgs', 'liblinear'],
    'max_iter': [1000000]
}

logreg = LogisticRegression()
clf = GridSearchCV(logreg,                    # model
                   param_grid = parameters,   # hyperparameters
                   scoring=rcl_scorer,        # metric for scoring ---> used to be 'recall', does not work
                   cv=10)                     # number of folds         worked with 'accuracy'



In [50]:
clf.fit(X_train,y_train)

C:\Users\Mike\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:1323: UserWarning: Setting penalty='none' will ignore the C and l1_ratio parameters
  "Setting penalty='none' will ignore the C and l1_ratio "
C:\Users\Mike\anaconda3\lib\site-packages\sklearn\model_selection\_validation.py:700: UserWarning: Scoring failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "C:\Users\Mike\anaconda3\lib\site-packages\sklearn\model_selection\_validation.py", line 687, in _score
    scores = scorer(estimator, X_test, y_test)
  File "C:\Users\Mike\anaconda3\lib\site-packages\sklearn\metrics\_scorer.py", line 200, in __call__
    sample_weight=sample_weight)
  File "C:\Users\Mike\anaconda3\lib\site-packages\sklearn\metrics\_scorer.py", line 243, in _score
    **self._kwargs)
  File "C:\Users\Mike\anaconda3\lib\site-packages\sklearn\utils\validation.py", line 63, in inner_f
    return f(*args, **kwargs)


C:\Users\Mike\anaconda3\lib\site-packages\sklearn\model_selection\_validation.py:700: UserWarning: Scoring failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "C:\Users\Mike\anaconda3\lib\site-packages\sklearn\model_selection\_validation.py", line 687, in _score
    scores = scorer(estimator, X_test, y_test)
  File "C:\Users\Mike\anaconda3\lib\site-packages\sklearn\metrics\_scorer.py", line 200, in __call__
    sample_weight=sample_weight)
  File "C:\Users\Mike\anaconda3\lib\site-packages\sklearn\metrics\_scorer.py", line 243, in _score
    **self._kwargs)
  File "C:\Users\Mike\anaconda3\lib\site-packages\sklearn\utils\validation.py", line 63, in inner_f
    return f(*args, **kwargs)
  File "C:\Users\Mike\anaconda3\lib\site-packages\sklearn\metrics\_classification.py", line 1780, in recall_score
    zero_division=zero_division)
  File "C:\Users\Mike\anaconda3\lib\site-packages\sklearn\utils\validati

C:\Users\Mike\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:1323: UserWarning: Setting penalty='none' will ignore the C and l1_ratio parameters
  "Setting penalty='none' will ignore the C and l1_ratio "
C:\Users\Mike\anaconda3\lib\site-packages\sklearn\model_selection\_validation.py:700: UserWarning: Scoring failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "C:\Users\Mike\anaconda3\lib\site-packages\sklearn\model_selection\_validation.py", line 687, in _score
    scores = scorer(estimator, X_test, y_test)
  File "C:\Users\Mike\anaconda3\lib\site-packages\sklearn\metrics\_scorer.py", line 200, in __call__
    sample_weight=sample_weight)
  File "C:\Users\Mike\anaconda3\lib\site-packages\sklearn\metrics\_scorer.py", line 243, in _score
    **self._kwargs)
  File "C:\Users\Mike\anaconda3\lib\site-packages\sklearn\utils\validation.py", line 63, in inner_f
    return f(*args, **kwargs)


C:\Users\Mike\anaconda3\lib\site-packages\sklearn\model_selection\_validation.py:700: UserWarning: Scoring failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "C:\Users\Mike\anaconda3\lib\site-packages\sklearn\model_selection\_validation.py", line 687, in _score
    scores = scorer(estimator, X_test, y_test)
  File "C:\Users\Mike\anaconda3\lib\site-packages\sklearn\metrics\_scorer.py", line 200, in __call__
    sample_weight=sample_weight)
  File "C:\Users\Mike\anaconda3\lib\site-packages\sklearn\metrics\_scorer.py", line 243, in _score
    **self._kwargs)
  File "C:\Users\Mike\anaconda3\lib\site-packages\sklearn\utils\validation.py", line 63, in inner_f
    return f(*args, **kwargs)
  File "C:\Users\Mike\anaconda3\lib\site-packages\sklearn\metrics\_classification.py", line 1780, in recall_score
    zero_division=zero_division)
  File "C:\Users\Mike\anaconda3\lib\site-packages\sklearn\utils\validati

C:\Users\Mike\anaconda3\lib\site-packages\sklearn\model_selection\_validation.py:700: UserWarning: Scoring failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "C:\Users\Mike\anaconda3\lib\site-packages\sklearn\model_selection\_validation.py", line 687, in _score
    scores = scorer(estimator, X_test, y_test)
  File "C:\Users\Mike\anaconda3\lib\site-packages\sklearn\metrics\_scorer.py", line 200, in __call__
    sample_weight=sample_weight)
  File "C:\Users\Mike\anaconda3\lib\site-packages\sklearn\metrics\_scorer.py", line 243, in _score
    **self._kwargs)
  File "C:\Users\Mike\anaconda3\lib\site-packages\sklearn\utils\validation.py", line 63, in inner_f
    return f(*args, **kwargs)
  File "C:\Users\Mike\anaconda3\lib\site-packages\sklearn\metrics\_classification.py", line 1780, in recall_score
    zero_division=zero_division)
  File "C:\Users\Mike\anaconda3\lib\site-packages\sklearn\utils\validati

...

C:\Users\Mike\anaconda3\lib\site-packages\sklearn\model_selection\_validation.py:700: UserWarning: Scoring failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "C:\Users\Mike\anaconda3\lib\site-packages\sklearn\model_selection\_validation.py", line 687, in _score
    scores = scorer(estimator, X_test, y_test)
  File "C:\Users\Mike\anaconda3\lib\site-packages\sklearn\metrics\_scorer.py", line 200, in __call__
    sample_weight=sample_weight)
  File "C:\Users\Mike\anaconda3\lib\site-packages\sklearn\metrics\_scorer.py", line 243, in _score
    **self._kwargs)
  File "C:\Users\Mike\anaconda3\lib\site-packages\sklearn\utils\validation.py", line 63, in inner_f
    return f(*args, **kwargs)
  File "C:\Users\Mike\anaconda3\lib\site-packages\sklearn\metrics\_classification.py", line 1780, in recall_score
    zero_division=zero_division)
  File "C:\Users\Mike\anaconda3\lib\site-packages\sklearn\utils\validati

C:\Users\Mike\anaconda3\lib\site-packages\sklearn\model_selection\_validation.py:700: UserWarning: Scoring failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "C:\Users\Mike\anaconda3\lib\site-packages\sklearn\model_selection\_validation.py", line 687, in _score
    scores = scorer(estimator, X_test, y_test)
  File "C:\Users\Mike\anaconda3\lib\site-packages\sklearn\metrics\_scorer.py", line 200, in __call__
    sample_weight=sample_weight)
  File "C:\Users\Mike\anaconda3\lib\site-packages\sklearn\metrics\_scorer.py", line 243, in _score
    **self._kwargs)
  File "C:\Users\Mike\anaconda3\lib\site-packages\sklearn\utils\validation.py", line 63, in inner_f
    return f(*args, **kwargs)
  File "C:\Users\Mike\anaconda3\lib\site-packages\sklearn\metrics\_classification.py", line 1780, in recall_score
    zero_division=zero_division)
  File "C:\Users\Mike\anaconda3\lib\site-packages\sklearn\utils\validati

C:\Users\Mike\anaconda3\lib\site-packages\sklearn\model_selection\_validation.py:619: FitFailedWarning: Estimator fit failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "C:\Users\Mike\anaconda3\lib\site-packages\sklearn\model_selection\_validation.py", line 598, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "C:\Users\Mike\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py", line 1306, in fit
    solver = _check_solver(self.solver, self.penalty, self.dual)
  File "C:\Users\Mike\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py", line 444, in _check_solver
    "got %s penalty." % (solver, penalty))
ValueError: Solver newton-cg supports only 'l2' or 'none' penalties, got l1 penalty.

  FitFailedWarning)
C:\Users\Mike\anaconda3\lib\site-packages\sklearn\model_selection\_validation.py:619: FitFailedWarning: Estimator fit failed. The score on this train

C:\Users\Mike\anaconda3\lib\site-packages\sklearn\model_selection\_validation.py:700: UserWarning: Scoring failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "C:\Users\Mike\anaconda3\lib\site-packages\sklearn\model_selection\_validation.py", line 687, in _score
    scores = scorer(estimator, X_test, y_test)
  File "C:\Users\Mike\anaconda3\lib\site-packages\sklearn\metrics\_scorer.py", line 200, in __call__
    sample_weight=sample_weight)
  File "C:\Users\Mike\anaconda3\lib\site-packages\sklearn\metrics\_scorer.py", line 243, in _score
    **self._kwargs)
  File "C:\Users\Mike\anaconda3\lib\site-packages\sklearn\utils\validation.py", line 63, in inner_f
    return f(*args, **kwargs)
  File "C:\Users\Mike\anaconda3\lib\site-packages\sklearn\metrics\_classification.py", line 1780, in recall_score
    zero_division=zero_division)
  File "C:\Users\Mike\anaconda3\lib\site-packages\sklearn\utils\validati

GridSearchCV(cv=10, estimator=LogisticRegression(),
             param_grid={'C': array([1.e-03, 1.e-02, 1.e-01, 1.e+00, 1.e+01, 1.e+02, 1.e+03]),
                         'max_iter': [1000000], 'penalty': ['none', 'l2', 'l1'],
                         'solver': ['newton-cg', 'lbfgs', 'liblinear']},
             scoring=make_scorer(recall_score))

In [51]:
print("Tuned Hyperparameters :", clf.best_params_)
print("Recall :",clf.best_score_)

Tuned Hyperparameters : {'C': 0.001, 'max_iter': 1000000, 'penalty': 'none', 'solver': 'newton-cg'}
Recall : nan


In [52]:
logreg = LogisticRegression(C = 0.001, 
                            max_iter=1000000,
                            penalty = 'none', 
                            solver = 'newton-cg')

y_pred = logreg.fit(X_train,y_train).predict(X_test)
# y_pred = logreg.predict(X_test)
print("Recall:",metrics.recall_score(y_test,y_pred, average=None))
print(confusion_matrix(y_test, y_pred))
print(classification_report(y_test,y_pred))

Recall: [0.6015625  0.95670391]
[[ 231  153]
 [  93 2055]]
                   precision    recall  f1-score   support

Attrited Customer       0.71      0.60      0.65       384
Existing Customer       0.93      0.96      0.94      2148

         accuracy                           0.90      2532
        macro avg       0.82      0.78      0.80      2532
     weighted avg       0.90      0.90      0.90      2532



C:\Users\Mike\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:1323: UserWarning: Setting penalty='none' will ignore the C and l1_ratio parameters
  "Setting penalty='none' will ignore the C and l1_ratio "
